In [0]:
#@title Mount drive to colab
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
#@title Imports
import os
import numpy as np

dirname = "/content/gdrive/My Drive/BIL4112/"

In [3]:
# @title One Time File Read
# Unzip the files
"""
!unzip "/content/gdrive/My Drive/BIL4112/BIL4112-HW1.zip" -d "/content/gdrive/My Drive/BIL4112/"
"""

# Function to read the files and write them to arrays
"""
from PIL import Image

def readdata(foldername):
    inputs, outputs = [], []
    for i in range(10):
        for file in os.listdir(foldername + str(i)):
            filename = os.fsdecode(file)
            if filename.endswith(".png"):
                with Image.open(os.path.join(foldername + str(i), filename)) as im:
                    # TODO
                    # TEST WITHOUT
                    im = im.convert('1')
                    # TODO
                    # TEST WITHOUT
                    inputs.append(list(im.getdata()))
                    outputs.append(i)
    return np.array(inputs, dtype = np.uint8), np.array(outputs, dtype = np.uint8)
"""

# Actual reading
"""
x_train, y_train = readdata(dirname + "mnist-train/")
x_train[x_train == 255] = 1
x_test, y_test = readdata(dirname + "mnist-test/")
x_test[x_test == 255] = 1
"""


# Write arrays to files
"""
import _pickle as cPickle

train = list(zip(x_train, y_train))
test = list(zip(x_test, y_test))

with open(dirname + "train", 'wb') as outfile:
    cPickle.dump(train, outfile)
with open(dirname + "test", 'wb') as outfile:
    cPickle.dump(test, outfile)
"""

""""""

''

# Functions

In [0]:
#@title Function to show an image

from PIL import Image
from IPython import display
from io import BytesIO
def show_image(array, text):
    # https://stackoverflow.com/a/22902116
    with Image.fromarray((array.reshape((28,28)) * 255).astype('uint8')) as im:
        bio = BytesIO()
        im.save(bio, format='png')
        display.display(display.Image(bio.getvalue(), format='png', retina=True))
        print(text)

In [0]:
#@title Function to find the k closest entries
def find_closest_k(matrix, query, k):
    d = ((matrix - query)**2).sum(axis=1)**0.5  # compute distances
    ndx = d.argsort() # indirect sort 
    # print 10 nearest points to the chosen one
    return d[ndx[:k]], ndx[:k]

In [0]:
#@title Function to shuffle

# Shuffle train and test data
def shuffle_data(x, y):
    data = list(zip(x, y))
    
    from random import shuffle
    shuffle(data)
    
    x, y = zip(*data)
    x, y = np.array(x, dtype = float), np.array(y, dtype = np.uint8)
    return x, y

In [0]:
#@title Function to read train & test with pickle

# Read the arrays from files
def read_pickle(black_white):
    import _pickle as cPickle
    if black_white:
        filename = ""
    else:
        filename = "255norm"
    with open(dirname + "train" + filename, 'rb') as infile:
        train = cPickle.load(infile)
    with open(dirname + "test" + filename, 'rb') as infile:
        test = cPickle.load(infile)


    x_train, y_train = zip(*train)
    x_train, y_train = np.array(x_train, dtype = float), np.array(y_train, dtype = np.uint8)
    x_test, y_test = zip(*test)
    x_test, y_test = np.array(x_test, dtype = float), np.array(y_test, dtype = np.uint8)
    
    return x_train, y_train, x_test, y_test

In [0]:
#@title Function to write outputs to csv
def write_to_file(file, text, result):
    acc, aver = result
    file.write("Accuracy değeri " + text + ";")
    for i in range(len(acc)):
        file.write(str(acc[i]) + ";")
    file.write(str(aver) + "\n")

In [0]:
#@title Functions to create codebooks
"""
# CODEBOOK CREATION WITH MEAN
def create_codebook_with_mean(x_train, y_train):
    codebook = []
    for i in range(10):
        codebook.append(np.mean(x_train[i*200:(i+1)*200], axis=0))
    codebook = np.array(codebook, dtype = float)
    
    x_train, y_train = shuffle_data(x_train, y_train)    
    return x_train, y_train, codebook
"""
# CODEBOOK CREATION WITH FIRST
def create_codebook_with_first(x_train, y_train, n):
    codebook = []
    y_codebook = []
    indices = []
    for i in range(10):
        for j in range(n):
            codebook.append(x_train[i*200 + n])
            y_codebook.append(i)
            indices.append(i*200 + n)
    x_train = np.delete(x_train, indices, 0)
    y_train = np.delete(y_train, indices, 0)
    codebook = np.array(codebook, dtype = float)
    y_codebook = np.array(y_codebook, dtype = np.uint8)
    
    x_train, y_train = shuffle_data(x_train, y_train)    
    return x_train, y_train, codebook, y_codebook

# CODEBOOK CREATION WITH SHUFFLED FIRST
def create_codebook_with_shuffled_first(x_train, y_train, n):
    codebook = []
    y_codebook = []
    indices = []
    x_train, y_train = shuffle_data(x_train, y_train)    
    
    
    for i in range(10):
        k = 0
        for j in range(len(y_train)):
            if i == y_train[j] and k < n:
                codebook.append(x_train[j])
                y_codebook.append(i)
                indices.append(j)
                k += 1
            elif k >= n:
                break
                
    x_train = np.delete(x_train, indices, 0)
    y_train = np.delete(y_train, indices, 0)
    codebook = np.array(codebook, dtype = float)
    y_codebook = np.array(y_codebook, dtype = np.uint8)
    
    return x_train, y_train, codebook, y_codebook

# Training

In [0]:
#@title LVQ 1 Train - Test

black_white = False #@param {type:"boolean"}

prototypes_per_class = 10 #@param {type:"slider", min:1, max:100, step:1}

x_train, y_train, x_test, y_test = read_pickle(black_white)
x_train, y_train, codebook, y_codebook = \
create_codebook_with_shuffled_first(x_train, y_train, prototypes_per_class)

learning_rate_start = 0.1 #@param {type:"slider", min:0, max:1, step:0.01}

learning_rate_step = True #@param {type:"boolean"}
learning_rate_step_size = -0.001 #@param {type:"slider", min:-0.5, max:0, step:0.0001}
number_of_epochs = 100 #@param {type:"slider", min:1, max:5000, step:1}

learning_rate = learning_rate_start
k = 1
#for learning_rate in np.arange(learning_rate_start, 0.01, -0.001):
for epoch in range(1, number_of_epochs + 1):
    for i in range(len(y_train)):
        _, idx = find_closest_k(codebook, x_train[i], k)

        if y_train[i] == y_codebook[idx]:
            codebook[idx] += learning_rate * (x_train[i] - codebook[idx])
        else:
            codebook[idx] -= learning_rate * (x_train[i] - codebook[idx])
    if learning_rate_step:
        learning_rate += learning_rate_step_size
k = 1
true = [0] * 10
false = [0] * 10
for i in range(len(y_test)):
    _, idx = find_closest_k(codebook, x_test[i], k)

    if y_test[i] == y_codebook[idx]:
        true[y_test[i]] += 1
    else:
        false[y_test[i]] += 1
        show_image(x_test[i], "Truth = {}\nPrediction = {}\n=====".format(y_test[i], y_codebook[idx][0]))

print("===LVQ1===")
print("Epochs: " + str(epoch))
acc = [t/(t + f) for (t , f) in zip(true, false)]
print("True:")
print(true)
print("False:")
print(false)
print("Acc:")
print(acc)
aver = sum(acc) / len(acc)
print("Average: " + str(aver))
lvq1result = (acc, aver)

In [18]:
#@title LVQ 2.1 Train - Test

black_white = False #@param {type:"boolean"}

prototypes_per_class = 5 #@param {type:"slider", min:1, max:100, step:1}

x_train, y_train, x_test, y_test = read_pickle(black_white)
x_train, y_train, codebook, y_codebook = \
create_codebook_with_shuffled_first(x_train, y_train, prototypes_per_class)

learning_rate_start = 0.1 #@param {type:"slider", min:0, max:1, step:0.01}
learning_rate_step = True #@param {type:"boolean"}
learning_rate_step_size = -0.001 #@param {type:"slider", min:-0.5, max:0, step:0.0001}

relative_window = 0.25 #@param {type:"slider", min:0, max:1, step:0.01}
number_of_epochs = 57 #@param {type:"slider", min:1, max:5000, step:1}

learning_rate = learning_rate_start

s = (1 - relative_window) / (1 + relative_window)
k = 2
for epoch in range(1, number_of_epochs + 1):
    for i in range(len(y_train)):
        dist, idx = find_closest_k(codebook, x_train[i], k)

        distance = min(dist[0] / dist[1], dist[1] / dist[0])
        if distance > s and ( \
            (y_codebook[idx[0]] == y_train[i] and y_codebook[idx[1]] != y_train[i]) or \
            (y_codebook[idx[0]] != y_train[i] and y_codebook[idx[1]] == y_train[i])):
            if y_train[i] == y_codebook[idx[0]]:
                codebook[idx[0]] += learning_rate * (x_train[i] - codebook[idx[0]])
                codebook[idx[1]] -= learning_rate * (x_train[i] - codebook[idx[1]])
            else:
                codebook[idx[0]] -= learning_rate * (x_train[i] - codebook[idx[0]])
                codebook[idx[1]] += learning_rate * (x_train[i] - codebook[idx[1]])
        elif y_train[i] == y_codebook[idx[0]]:
            codebook[idx[0]] += learning_rate * (x_train[i] - codebook[idx[0]])
        else:
            codebook[idx[0]] -= learning_rate * (x_train[i] - codebook[idx[0]])
    if learning_rate_step:
        learning_rate += learning_rate_step_size

k = 1
true = [0] * 10
false = [0] * 10
for i in range(len(y_test)):
    _, idx = find_closest_k(codebook, x_test[i], k)

    if y_test[i] == y_codebook[idx]:
        true[y_test[i]] += 1
    else:
        false[y_test[i]] += 1
        show_image(x_test[i], "Truth = {}\nPrediction = {}\n=====".format(y_test[i], y_codebook[idx][0]))

print("===LVQ2.1===")
print("Epochs: " + str(epoch))
acc = [t/(t + f) for (t , f) in zip(true, false)]
print("True:")
print(true)
print("False:")
print(false)
print("Acc:")
print(acc)
aver = sum(acc) / len(acc)
print("Average: " + str(aver))
lvq21result = (acc, aver)

/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in square
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: RuntimeWarning: overflow encountered in subtract
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in add


Truth = 0
Prediction = 3
=====


Truth = 0
Prediction = 3
=====


Truth = 0
Prediction = 3
=====


Truth = 0
Prediction = 3
=====


...

Truth = 9
Prediction = 3
=====


Truth = 9
Prediction = 3
=====


Truth = 9
Prediction = 3
=====


Truth = 9
Prediction = 3
=====


Truth = 9
Prediction = 3
=====
===LVQ2.1===
Epochs: 57
True:
[0, 0, 0, 100, 0, 0, 0, 0, 0, 0]
False:
[100, 100, 100, 0, 100, 100, 100, 100, 100, 100]
Acc:
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Average: 0.1


In [0]:
#@title LVQ 3 Train - Test

black_white = False #@param {type:"boolean"}

prototypes_per_class = 10 #@param {type:"slider", min:1, max:100, step:1}

x_train, y_train, x_test, y_test = read_pickle(black_white)
x_train, y_train, codebook, y_codebook = \
create_codebook_with_shuffled_first(x_train, y_train, prototypes_per_class)

learning_rate_start = 0.1 #@param {type:"slider", min:0, max:1, step:0.01}
learning_rate_step = True #@param {type:"boolean"}
learning_rate_step_size = -0.001 #@param {type:"slider", min:-0.5, max:0, step:0.0001}
epsilon = 0.3 #@param {type:"slider", min:0.1, max:0.5, step:0.01}

relative_window = 0.25 #@param {type:"slider", min:0, max:1, step:0.01}
number_of_epochs = 100 #@param {type:"slider", min:1, max:5000, step:1}

learning_rate = learning_rate_start

s = (1 - relative_window) / (1 + relative_window)
k = 2
for epoch in range(1, number_of_epochs + 1):
    for i in range(len(y_train)):
        dist, idx = find_closest_k(codebook, x_train[i], k)

        distance = min(dist[0] / dist[1], dist[1] / dist[0])
        if distance > s and ( \
            (y_codebook[idx[0]] == y_train[i] and y_codebook[idx[1]] != y_train[i]) or \
            (y_codebook[idx[0]] != y_train[i] and y_codebook[idx[1]] == y_train[i])):
            if y_train[i] == y_codebook[idx[0]]:
                codebook[idx[0]] += learning_rate * (x_train[i] - codebook[idx[0]])
                codebook[idx[1]] -= learning_rate * (x_train[i] - codebook[idx[1]])
            else:
                codebook[idx[0]] -= learning_rate * (x_train[i] - codebook[idx[0]])
                codebook[idx[1]] += learning_rate * (x_train[i] - codebook[idx[1]])
        elif distance > s and \
            y_train[i] == y_codebook[idx[0]] and y_train[i] == y_codebook[idx[1]]:
            codebook[idx[0]] += learning_rate * epsilon * (x_train[i] - codebook[idx[0]])
            codebook[idx[1]] += learning_rate * epsilon * (x_train[i] - codebook[idx[1]])
        else:
            codebook[idx[0]] -= learning_rate * (x_train[i] - codebook[idx[0]])
    if learning_rate_step:
        learning_rate += learning_rate_step_size

k = 1
true = [0] * 10
false = [0] * 10
for i in range(len(y_test)):
    _, idx = find_closest_k(codebook, x_test[i], k)

    if y_test[i] == y_codebook[idx]:
        true[y_test[i]] += 1
    else:
        false[y_test[i]] += 1
        show_image(x_test[i], "Truth = {}\nPrediction = {}\n=====".format(y_test[i], y_codebook[idx][0]))

print("===LVQ3===")
print("Epochs: " + str(epoch))
acc = [t/(t + f) for (t , f) in zip(true, false)]
print("True:")
print(true)
print("False:")
print(false)
print("Acc:")
print(acc)
aver = sum(acc) / len(acc)
print("Average: " + str(aver))
lvq3result = (acc, aver)

In [0]:
#@title Write results to file
with open(dirname + "output.csv", 'w', encoding='utf8') as output_file:
    output_file.write("Rakam sınıfı;0;1;2;3;4;5;6;7;8;9;Ortalama\n")
    write_to_file(output_file, "LVQ1", lvq1result)
    write_to_file(output_file, "LVQ2.1", lvq21result)
    write_to_file(output_file, "LVQ3", lvq3result)